In [10]:
# import packages
from utils.download import download_dataset
import zipfile
import glob
from PIL import Image
import numpy as np
from main import MitoReID
import argparse
import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline
mpl.rcParams['figure.dpi'] = 300

In [ ]:
# Download the Mito dataset in zip format from Google drive.
download_dataset(name='Mito.zip', save_folder='./data')

# unzip Mito.zip.
with zipfile.ZipFile('./data/Mito.zip', 'r') as zip_ref:
    zip_ref.extractall('./data')

In [2]:
'''
The dataset's folder structure is as follows. Notably, l1, l2, and l3 represent three replicated experiments, each sharing an identical folder structure.

- l1: experiment 1
-- train: data for training.
-- query: data for evaluation, query and gallery are used together to evaluate.
-- gallery: data for evaluation, query and gallery are used together to evaluate.
-- nc: natural compounds.
- l2: experiment 2
-- (same as l1)
- l3: experiment 3
-- (same as l1)
- novel: unseen novel FDA-approved drugs.
- annotation.xlsx: annotations for the dataset. There are two sheets in the excel: 'FDA-list' for FDA approved drugs, 'Novel-list' for novel compounds.
'''

# view one image sequence
folder = glob.glob('./data/Mito/*/*/*/*')[0]
for i in range(1, 17):
    path = f'{folder}/{i}.jpg'
    img = np.array(Image.open(path))
    plt.subplot(4, 4, i)
    plt.axis('off')
    plt.imshow(img)
plt.show()

NameError: name 'glob' is not defined

In [1]:
'''
Use the trained model to test on the query set. Configurations can be refered to `configs/test.yml`. Parameters explanation of the configurations can be found in main.py. Evaluation metrics are accuracy, rank-k and mAP. The model weight `mitoreid-resnet-50-pretrain` will be downloaded automatically for testing and predicting. All the model weight you can download are listed below. They all trained with I3D ResNet network.

Note: The standard I3D ResNet network downscales the temporal dimension throughout all stages while the modified I3D ResNet network only downscales the temporal dimension in the first two stages.

- resnet-18-kinetics: weight trained with standard I3D ResNet-18 network on Kinetics-400 dataset.
- resnet-50-kinetics: weight trained with standard I3D ResNet-50 network on Kinetics-400 dataset.
- resnet-18-mito: weight trained with modified I3D ResNet-18 network on Mito dataset using self-supervised pretraining technique.
- resnet-50-mito: weight trained with modified I3D ResNet-50 network on Mito dataset using self-supervised pretraining technique.
- ide: weight trained with IDE framework with modified I3D ResNet-18 network on Mito dataset, pretrained from resnet-18-kinetics.
- mitoreid-resnet-18: weight trained with MitoReID framework with modified I3D ResNet-18 network on Mito dataset, pretrained from resnet-18-kinetics.
- mitoreid-resnet-50: weight trained with MitoReID framework with modified I3D ResNet-50 network on Mito dataset, pretrained from resnet-50-kinetics.
- mitoreid-resnet-18-standard: weight trained with MitoReID framework with standard I3D ResNet-18 network on Mito dataset, pretrained from resnet-18-kinetics.
- mitoreid-resnet-50-standard: weight trained with MitoReID framework with standard I3D ResNet-50 network on Mito dataset, pretrained from resnet-50-kinetics.
- mitoreid-resnet-18-image: weight trained with MitoReID framework with standard I3D ResNet-18 network on Mito dataset (image only), pretrained from resnet-18-kinetics.
- mitoreid-resnet-50-image: weight trained with MitoReID framework with standard I3D ResNet-50 network on Mito dataset (image only), pretrained from resnet-50-kinetics.
- mitoreid-resnet-18-pretrain: weight trained with MitoReID framework with modified I3D ResNet-18 network on Mito dataset, pretrained from resnet-18-mito.
- mitoreid-resnet-50-pretrain: weight trained with MitoReID framework with modified I3D ResNet-50 network on Mito dataset, pretrained from resnet-50-mito.
'''

mito = MitoReID('configs/test.yml')
mito.test()

c:\users\weiming\appdata\local\programs\python\python38\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Namespace(arch='resnet50', augment=1, bnneck=1, bs=128, center_lr=0.05, choose_col='moa_choose', config='configs/test.yml', device_ids='0,1', downscale_temporal_dimension=0, epoch=400, feature_path='', freeze=0, gallery_path='/home/weiming.li/dataset/mito_center/l1/gallery,/home/weiming.li/dataset/mito_center/l2/gallery,/home/weiming.li/dataset/mito_center/l3/gallery', id_col='moa_id', input_size=112, loss='id+trihard', lr=0.0001, lr_patience=5, margin=0.1, mean=[0.023, 0.06, 0.015], mode='test', model_path='result/mitosnet_resnet50_size_112_pretrain_37_base_train_gallery_bak.pth', num_classes=37, num_instances=4, num_mislabel=5, num_timepoint=16, num_workers=16, optimizer='sgd', predict_path='', query_path='/home/weiming.li/dataset/mito_center/l1/query,/home/weiming.li/dataset/mito_center/l2/query,/home/weiming.li/dataset/mito_center/l3/query', resume=1, resume_path='result/mitosnet_resnet50_size_112_pretrain_37_base_train_gallery_bak_best.pth', sample_rate=1, sample_size=128, schedul

In [ ]:
'''
Use the trained model to predict the natural compounds' MOAs. Configurations can be refered to `configs/predict.yml`. You are expected to get the predicted MOA id (pred), score (rank1 and cosine_distance) and the name of the MOA (pred_name).
'''

mito = MitoReID('configs/predict.yml')
mito.predict()

In [ ]:
'''
Fine-tune your model based on the self-supervised pretrained weight. Configurations can be refered to `configs/fine-tune.yml`. The pretrained weight `resnet-50-mito` will be downloaded automatically. Train on V100 will take about 1 hour. Wandb is used to visualize the training process, register the wandb account in `https://wandb.ai/site` if you don't have.
'''

# fine-tune your model based on the pretrained weight
mito = MitoReID('configs/fine-tune.yml')
mito.train()

In [ ]:
'''
Pretrain your model using self-supervised technique. Configurations can be refered to configs/pretrain.yml. The pretraining is based on the weight trained on Kinetics-400 dataset, which is benchmark for video classification.The pretrained weight `resnet-50-kinetics` will be downloaded automatically. Train on V100 will take about 4 hours.

Note: Train from scratch is not recommended considering the difficulty of image sequences classification. Too much training may led to overfitting, don’t set too large epoch in the config.
'''

# pretrain your model using self-supervised technique
mito = MitoReID('configs/pretrain.yml')
mito.train()

In [ ]:
'''
Train your model from scratch base on the weight trained on Kinetics-400 dataset. Configurations can be refered to `configs/train.yml`. The pretrained weight `resnet-50-kinetics` will be downloaded automatically. Train on V100 will take about 5 hours.
'''

# train your model from scratch
mito = MitoReID('configs/train.yml')
mito.train()